<a href="https://colab.research.google.com/github/N3wBaz/MLzoomcamp/blob/main/hw3/homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
import pandas as pd
import numpy as np

In [116]:
df = pd.read_csv('housing.csv')

In [117]:
df.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [118]:
df.fillna(0, inplace=True)

In [119]:
df['rooms_per_household'] = df['total_rooms'] / df['households']

In [120]:
df['bedrooms_per_room'] = df['total_bedrooms'] / df['total_rooms']

In [121]:
df['population_per_household'] = df['population'] / df['households']

In [122]:
df['ocean_proximity'].mode()

0    <1H OCEAN
dtype: object

In [ ]:
# for i in df.columns:
# #     print(i)
#     if i == 'ocean_proximity' or i == 'longitude':
#         continue
#     print(f"{(df.corr()[i]).sort_values()[-2:-1]}")

In [123]:
(df.corr()['total_bedrooms']).sort_values()

housing_median_age         -0.317063
latitude                   -0.065318
population_per_household   -0.028019
median_income              -0.007295
rooms_per_household         0.002717
median_house_value          0.049148
longitude                   0.068082
bedrooms_per_room           0.122205
population                  0.866266
total_rooms                 0.920196
households                  0.966507
total_bedrooms              1.000000
Name: total_bedrooms, dtype: float64

In [124]:
(df.corr()['population']).sort_values()

housing_median_age         -0.296244
latitude                   -0.108785
rooms_per_household        -0.072213
median_house_value         -0.024650
median_income               0.004834
bedrooms_per_room           0.031397
population_per_household    0.069863
longitude                   0.099773
total_rooms                 0.857126
total_bedrooms              0.866266
households                  0.907222
population                  1.000000
Name: population, dtype: float64

In [126]:
(df.corr()['population_per_household']).sort_values()

total_bedrooms             -0.028019
households                 -0.027309
total_rooms                -0.024581
median_house_value         -0.023737
rooms_per_household        -0.004852
latitude                    0.002366
longitude                   0.002476
bedrooms_per_room           0.003047
housing_median_age          0.013191
median_income               0.018766
population                  0.069863
population_per_household    1.000000
Name: population_per_household, dtype: float64

In [127]:
def above_median(series):
    
    return 1 if series.median() > series else 0
# df[]np.where(df['median_house_value'] > df['median_house_value'])

In [128]:
df['above_average'] = (df['median_house_value'].mean() > df['median_house_value']).astype(int)

In [129]:
df['above_average']

0        0
1        0
2        0
3        0
4        0
        ..
20635    1
20636    1
20637    1
20638    1
20639    1
Name: above_average, Length: 20640, dtype: int64

In [131]:
from sklearn.model_selection import train_test_split

In [138]:
SEED = 42
df_train, df_test_val = train_test_split(df, test_size=0.4, random_state=SEED)
df_test, df_val = train_test_split(df_test_val, test_size=0.5, random_state=SEED)

In [139]:
del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

In [140]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)


y_train = df_train['above_average'].values
y_test = df_test['above_average'].values
y_val = df_val['above_average'].values




# y_train = y_train.reset_index(drop=True)



In [141]:
from sklearn.metrics import mutual_info_score

In [143]:
round(mutual_info_score(df_train['above_average'], df_train['ocean_proximity']), 3)

0.103

In [144]:
del df_train['above_average']
del df_val['above_average']
del df_test['above_average']

In [145]:
# from sklearn.preprocessing import OneHotEncoder

# one_hot_train = pd.get_dummies(df_train, columns = ['ocean_proximity'], drop_first=True)
# one_hot_val = pd.get_dummies(df_val, columns = ['ocean_proximity'], drop_first=True)
# one_hot_test = pd.get_dummies(df_test, columns = ['ocean_proximity'], drop_first=True)


In [146]:
# print(one_hot_train.columns)
# print(one_hot_val.columns)
# print(one_hot_test.columns)

In [147]:
# one_hot_train.info()

In [148]:
from sklearn.feature_extraction import DictVectorizer

In [170]:
numerical = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
             'total_bedrooms', 'population', 'households', 'median_income',
             'rooms_per_household', 'bedrooms_per_room', 'population_per_household']
categorical = ['ocean_proximity']

In [171]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[numerical + categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[numerical + categorical].to_dict(orient='records')
test_dict = df_test[numerical + categorical].to_dict(orient='records')
X_val = dv.transform(val_dict)
X_test = dv.transform(test_dict)

In [172]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_val = scaler.transform(X_val)
# X_test = scaler.transform(X_test)

In [173]:
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=100, random_state=SEED)
model.fit(X_train, y_train)
# y_train.shape

LogisticRegression(random_state=42, solver='liblinear')

In [174]:
model.predict_proba(X_train)

array([[0.96064227, 0.03935773],
       [0.00242616, 0.99757384],
       [0.04439927, 0.95560073],
       ...,
       [0.23438697, 0.76561303],
       [0.81331863, 0.18668137],
       [0.79141063, 0.20858937]])

In [175]:
from sklearn.metrics import accuracy_score

In [178]:
acc = accuracy_score(y_val, model.predict(X_val))
acc

0.8406007751937985

In [180]:
# without total_rooms
numerical = ['longitude', 'latitude', 'housing_median_age', 
             'total_bedrooms', 'population', 'households', 'median_income',
             'rooms_per_household', 'bedrooms_per_room', 'population_per_household']
categorical = ['ocean_proximity']

dv = DictVectorizer(sparse=False)

train_dict = df_train[numerical + categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[numerical + categorical].to_dict(orient='records')
# test_dict = df_test[numerical + categorical].to_dict(orient='records')
X_val = dv.transform(val_dict)
# X_test = dv.transform(test_dict)

model = LogisticRegression(solver="liblinear", C=1.0, max_iter=100, random_state=SEED)
model.fit(X_train, y_train)

total_rooms_acc = accuracy_score(y_val, model.predict(X_val))
total_rooms_acc

0.843265503875969

In [181]:
# without total_bedrooms
numerical = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
             'population', 'households', 'median_income',
             'rooms_per_household', 'bedrooms_per_room', 'population_per_household']
categorical = ['ocean_proximity']

dv = DictVectorizer(sparse=False)

train_dict = df_train[numerical + categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[numerical + categorical].to_dict(orient='records')
# test_dict = df_test[numerical + categorical].to_dict(orient='records')
X_val = dv.transform(val_dict)
# X_test = dv.transform(test_dict)

model = LogisticRegression(solver="liblinear", C=1.0, max_iter=100, random_state=SEED)
model.fit(X_train, y_train)

total_bedrooms_acc = accuracy_score(y_val, model.predict(X_val))
total_bedrooms_acc

0.8396317829457365

In [182]:
# without population
numerical = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
              'households', 'median_income', 'total_bedrooms',
             'rooms_per_household', 'bedrooms_per_room', 'population_per_household']
categorical = ['ocean_proximity']

dv = DictVectorizer(sparse=False)

train_dict = df_train[numerical + categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[numerical + categorical].to_dict(orient='records')
# test_dict = df_test[numerical + categorical].to_dict(orient='records')
X_val = dv.transform(val_dict)
# X_test = dv.transform(test_dict)

model = LogisticRegression(solver="liblinear", C=1.0, max_iter=100, random_state=SEED)
model.fit(X_train, y_train)

population_acc = accuracy_score(y_val, model.predict(X_val))
population_acc

0.8335755813953488

In [184]:
# without households
numerical = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
              'population', 'median_income', 'total_bedrooms',
             'rooms_per_household', 'bedrooms_per_room', 'population_per_household']
categorical = ['ocean_proximity']

dv = DictVectorizer(sparse=False)

train_dict = df_train[numerical + categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[numerical + categorical].to_dict(orient='records')
# test_dict = df_test[numerical + categorical].to_dict(orient='records')
X_val = dv.transform(val_dict)
# X_test = dv.transform(test_dict)

model = LogisticRegression(solver="liblinear", C=1.0, max_iter=100, random_state=SEED)
model.fit(X_train, y_train)

households_acc = accuracy_score(y_val, model.predict(X_val))
households_acc

0.8386627906976745

In [185]:
print(acc - households_acc)
print(acc - population_acc)
print(acc - total_bedrooms_acc)
print(acc - total_rooms_acc)

0.0019379844961240345
0.00702519379844968
0.0009689922480620172
-0.002664728682170492


In [189]:
SEED = 42
df_train, df_test_val = train_test_split(df, test_size=0.4, random_state=SEED)
df_test, df_val = train_test_split(df_test_val, test_size=0.5, random_state=SEED)

In [190]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)


y_train = np.log(df_train['median_house_value'].values)
y_test = np.log(df_test['median_house_value'].values)
y_val = np.log(df_val['median_house_value'].values)




# y_train = y_train.reset_index(drop=True)



In [191]:
del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

In [194]:
from sklearn.metrics import mean_squared_error

In [197]:
from sklearn.linear_model import Ridge

rmse = []
for i in [0, 0.01, 0.1, 1, 10]:
    ridge = Ridge(alpha=i)
    ridge.fit(X_train, y_train)
    rmse.append(round(mean_squared_error(y_val, ridge.predict(X_val), squared=False), 3))

In [198]:
rmse

[0.338, 0.339, 0.339, 0.339, 0.338]